In [5]:
import pandas as pd
import numpy as np

In [8]:
dataset_df = pd.read_csv(f"./contrasting_power_over.tsv", sep="\t")
output_pfname = "contrasting_power_over_prompts.txt"
output_lfname = "contrasting_power_over_power_labels.txt"

# Note that we add ___ after the end, this will be replaced with white space after the prompt is formatted
# For LLama. We do this because otherwise the last trailing space will be deleted by format_chat_prompt.
# Without the trailing whitespace, the model usually predicts " " rather than "YES" or "NO" as the next token.
# For some reason, if there is only one space between ":  ___" instead of two, the model predicts "'" as the top token
formatted_prompts = []
formatted_labels = [] # 1 if subject has power, 0 if subject does not have power
for (_, row_name, row_power_sent, row_no_power_sent) in dataset_df.itertuples(name=None):
    for sentence in [row_power_sent, row_no_power_sent]:
        formatted_prompt_start = f"Given the following sentence, determine if {row_name} has power over another person or is powerless (has power exercised over them).\n\nSentence: \"{sentence}\"\n\n"
        formatted_prompts.append(formatted_prompt_start + f"Does {row_name} have power over another person in this sentence?\nAnswer with only 'YES' if {row_name} has power over someone else, or 'NO' if {row_name} is powerless (someone has power over them).\n\nAnswer:  ___")
        formatted_prompts.append(formatted_prompt_start + f"Does another person have power over {row_name} in this sentence?\nAnswer with only 'NO' if {row_name} has power over someone else, or 'YES' if {row_name} is powerless (someone has power over them).\n\nAnswer:  ___")
    formatted_labels.extend([1, 1, 0, 0])


shuffled_idxs = np.random.choice(len(formatted_prompts), size=len(formatted_prompts), replace=False)
shuffled_fprompts = np.array(formatted_prompts, dtype=object)[shuffled_idxs]
shuffled_ylabels = np.array(formatted_labels)[shuffled_idxs]

# Write seperated by tabs to preserve spacing
with open(output_pfname, "w") as outfile:
    outfile.write("\t".join(shuffled_fprompts))

with open(output_lfname, "w") as outfile:
    outfile.write("\t".join(shuffled_ylabels.astype(str)))

print(f"Num prompts {len(formatted_prompts)}")

Num prompts 504
